In [ ]:
try:
    from google.colab import files
    uploaded = files.upload()

    import zipfile
    with zipfile.ZipFile("manual_cat_dataset.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    print("Dataset unzipped successfully!")
except:
    print("Running outside Google Colab or skipping file upload.")

Saving manual_cat_dataset.zip to manual_cat_dataset.zip
Dataset unzipped successfully!


In [ ]:
import os
import random
import numpy as np
import shutil
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn.metrics import classification_report
from glob import glob

In [ ]:
# MobileViT model from timm
!pip install -q timm
import timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00


In [ ]:
# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tfm = transforms.Compose([
    transforms.Resize((224, 224)), # resizes images to 224x224 pixels
    transforms.RandomHorizontalFlip(), # randomly flips and rotates images (for training data augmentation)
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(), # converts images to pytorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Class names
class_names = ['alarmed', 'angry', 'calm', 'pleased']

In [ ]:
# === Simple Dataset Split (70/15/15) ===
from glob import glob

src_root = "./manual_cat_dataset"
dst_root = "./split_cat_dataset"

for cls in os.listdir(src_root):
    imgs = glob(os.path.join(src_root, cls, "*.jpg"))
    random.shuffle(imgs)
    n = len(imgs)
    train_cut = int(0.7 * n)
    val_cut = int(0.85 * n)
    splits = {'train': imgs[:train_cut], 'valid': imgs[train_cut:val_cut], 'test': imgs[val_cut:]}
    for split, files in splits.items():
        split_dir = os.path.join(dst_root, split, cls)
        os.makedirs(split_dir, exist_ok=True)
        for f in files:
            shutil.copy(f, split_dir)
print("✅ Dataset split done: 70% train, 15% valid, 15% test")

✅ Dataset split done: 70% train, 15% valid, 15% test


In [ ]:
# Dataset paths
train_img_dir = os.path.join(dst_root, "train")
val_img_dir = os.path.join(dst_root, "valid")
test_img_dir = os.path.join(dst_root, "test")

In [ ]:
# Load datasets using ImageFolder
train_set = datasets.ImageFolder(root=train_img_dir, transform=tfm)
val_set = datasets.ImageFolder(root=val_img_dir, transform=tfm)
test_set = datasets.ImageFolder(root=test_img_dir, transform=tfm)

In [ ]:
# Load datasets
train_loader = DataLoader(datasets.ImageFolder(train_img_dir, transform=tfm), batch_size=16, shuffle=True)
val_loader = DataLoader(datasets.ImageFolder(val_img_dir, transform=tfm), batch_size=16)
test_loader = DataLoader(datasets.ImageFolder(test_img_dir, transform=tfm), batch_size=16)

In [ ]:
# Load pretrained MobileViT model
model = timm.create_model('mobilevit_s', pretrained=True, num_classes=4)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Training
for epoch in range(10):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(imgs), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss={total_loss:.4f}")

Epoch 1: Loss=78.2894
Epoch 2: Loss=70.3228
Epoch 3: Loss=66.2522
Epoch 4: Loss=61.6504
Epoch 5: Loss=56.3887
Epoch 6: Loss=51.3187
Epoch 7: Loss=48.6905
Epoch 8: Loss=45.5291
Epoch 9: Loss=41.4373
Epoch 10: Loss=40.2464


In [ ]:
# --- Simplified Validation (Accuracy and Loss) ---
if val_loader and val_set and len(val_set) > 0:
    print("\nCalculating Validation Metrics...")
    model.eval()
    correct = 0
    total = 0
    val_running_loss = 0.0 # Initialize validation loss
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Calculate validation loss for this batch
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)

            # Get predictions for accuracy calculation
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if total > 0:
        val_accuracy = 100 * correct / total
        val_epoch_loss = val_running_loss / len(val_set) # Calculate average validation loss
        print(f"Validation Loss: {val_epoch_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.2f}%")
    else:
        print("No data in validation loader to calculate metrics.")
else:
    print("Validation data not loaded or is empty. Skipping validation.")


Calculating Validation Metrics...
Validation Loss: 0.8436
Validation Accuracy: 65.84%


In [ ]:
# Test Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


Classification Report:
              precision    recall  f1-score   support

     alarmed       0.48      0.58      0.53        26
       angry       0.68      0.70      0.69        61
        calm       0.50      0.40      0.45        47
     pleased       0.81      0.83      0.82        70

    accuracy                           0.66       204
   macro avg       0.62      0.63      0.62       204
weighted avg       0.66      0.66      0.66       204

